# Pneumonia Detection Using Convolutional Neural Networks in TensorFlow

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow.keras.layers.experimental.preprocessing as preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Creating the pipelines

In [2]:
def process_img(img, label):
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    return img, label

train_ds = image_dataset_from_directory(
    '../input/chest-xray-pneumonia/chest_xray/train',
    labels='inferred',
    shuffle=True,
    batch_size=64,
    image_size=(256,256),
    color_mode='grayscale',
    label_mode='binary',
    interpolation='nearest'
).map(process_img)

test_ds = image_dataset_from_directory(
    '../input/chest-xray-pneumonia/chest_xray/test',
    labels='inferred',
    shuffle=False,
    batch_size=64,
    image_size=(256,256),
    color_mode='grayscale',
    label_mode='binary',
    interpolation='nearest'
).map(process_img)

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


## Making the model

In [3]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(256,256,1)),

    # Data Augmentation
    preprocessing.RandomContrast(0.3),

    # Body
    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),

    # Head
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compiling the model
I used the Adam optimizer with a learning rate of 1e-4 or 0.0001

In [4]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_contrast (RandomContr (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 128)       0

## Training the model

In [5]:
early_stopping = EarlyStopping(
    min_delta = 1e-3,
    patience=3,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    train_ds,
    validation_data=(test_ds),
    batch_size=64,
    epochs=10,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/10
82/82 [==============================] - 92s 1s/step - loss: 0.5361 - binary_accuracy: 0.7454 - val_loss: 0.4819 - val_binary_accuracy: 0.8173
Epoch 2/10
82/82 [==============================] - 48s 581ms/step - loss: 0.3015 - binary_accuracy: 0.8600 - val_loss: 0.4322 - val_binary_accuracy: 0.8189
Epoch 3/10
82/82 [==============================] - 46s 558ms/step - loss: 0.1929 - binary_accuracy: 0.9277 - val_loss: 0.4792 - val_binary_accuracy: 0.8253
Epoch 4/10
82/82 [==============================] - 45s 552ms/step - loss: 0.1750 - binary_accuracy: 0.9333 - val_loss: 0.6291 - val_binary_accuracy: 0.7853
Epoch 5/10
82/82 [==============================] - 46s 556ms/step - loss: 0.1482 - binary_accuracy: 0.9498 - val_loss: 0.7260 - val_binary_accuracy: 0.7821
Epoch 00005: early stopping


## Saving the model

In [6]:
model.save('pneumonia-model.h5')